# ICC ChampTrophy 2025 all Data

In [46]:
import numpy as np
import pandas as pd
import json
import http.client
import time

In [47]:
match_details = pd.DataFrame()
match_incidents = pd.DataFrame()
match_graph = pd.DataFrame()
match_stats = pd.DataFrame()
battingTeam = pd.DataFrame()
bowlingTeam = pd.DataFrame()
battingLine = pd.DataFrame()
bowlingLine = pd.DataFrame()
partnerships = pd.DataFrame()
match_lineups = pd.DataFrame()
match_h2h_duel = pd.DataFrame()
match_odds = pd.DataFrame()
head_to_head_match = pd.DataFrame()

In [ ]:
# Importing all essential Libraries
import numpy as np
import pandas as pd
import json
import http.client
import time

# Empty Dataframe to store each Data Table
match_details = pd.DataFrame()
match_incidents = pd.DataFrame()
match_graph = pd.DataFrame()
match_stats = pd.DataFrame()
battingTeam = pd.DataFrame()
bowlingTeam = pd.DataFrame()
battingLine = pd.DataFrame()
bowlingLine = pd.DataFrame()
partnerships = pd.DataFrame()
match_lineups = pd.DataFrame()
match_h2h_duel = pd.DataFrame()
match_odds = pd.DataFrame()
head_to_head_match = pd.DataFrame()

# **********************************************************************************************
match_ids = [13265827]



# Develop a code to collect all match_ids from tournament 


# ***********************************************************************

# Function for Convert a nested json dictionary to flatten json or dictionary*************************
def flatten_json(nested_json, parent_key='', sep='_'):

    items = []
    
    # If the value is a dictionary
    if isinstance(nested_json, dict):
        for key, value in nested_json.items():
            new_key = f"{parent_key}{sep}{key}" if parent_key else key
            items.extend(flatten_json(value, new_key, sep=sep).items())
    
    # If the value is a list
    elif isinstance(nested_json, list): 
        for index, item in enumerate(nested_json):
            new_key = f"{parent_key}{sep}{index}" if parent_key else str(index)
            items.extend(flatten_json(item, new_key, sep=sep).items())
            
    # If the value is neither dict nor list
    else:
        items.append((parent_key, nested_json))

    return dict(items)


# Function to convert  flatten_json to Row column **********************************
def flatten_json_to_row_column(data):
    try:
        # Normalize JSON data
        if isinstance(data, list):  # JSON is an array of objects
            flat_data = [flatten_json(item) for item in data]
        elif isinstance(data, dict):  # JSON is a single dictionary
            flat_data = [flatten_json(data)]
        else:
            raise ValueError("Unsupported JSON format")

        # Convert to DataFrame
        df = pd.DataFrame(flat_data)
        return df


    except json.JSONDecodeError:
        print("Error: Invalid JSON file.")
    except Exception as e:
        print(f"Error: {str(e)}")



# Main process code to convert json to row data********************************************
def process_json(data):
    all_dataframe = {}
    parent_json = {}

    for key in data:
        if data[key] is not None and data[key] != "" and data[key] != {}:
            if isinstance(data[key], list):  # If value is a list
                dataframe = pd.DataFrame(columns=flatten_json_to_row_column(flatten_json(data[key][0])).columns)
                for i in range(len(data[key])):
                    flatten_dict = flatten_json(data[key][i])
                    new_dataframe = flatten_json_to_row_column(flatten_dict)
                    dataframe = pd.concat([dataframe, new_dataframe], ignore_index=True)

                    # Deleting field translation columns
                    dataframe = dataframe.loc[:, ~dataframe.columns.str.contains("fieldtranslations|translation", case=False, regex=True)]

                if dataframe.shape[1] >= 20:  # If columns are 10 or more
                    all_dataframe[key] = dataframe
                else:
                    parent_json[key] = data[key]

            elif isinstance(data[key], dict):  # If value is a dictionary
                flatten_dict = flatten_json(data[key])
                dataframe = flatten_json_to_row_column(flatten_dict)

                # Deleting field translation columns
                dataframe = dataframe.loc[:, ~dataframe.columns.str.contains("fieldtranslations|translation", case=False, regex=True)]

                if dataframe.shape[1] >= 20:  # If columns are 5 or more
                    all_dataframe[key] = dataframe
                else:
                    parent_json[key] = data[key]
            else:
                parent_json[key] = data[key]

    # Convert parent_json into a DataFrame and store it in all_dataframe
    all_dataframe["parent_df"] = flatten_json_to_row_column(flatten_json(parent_json))

    return all_dataframe


# for match innings some special case handling 
def process_json_for_innings(data):
    all_dataframe = {}
    parent_json = {}

    for key in data:
        if data[key] is not None and data[key] != "" and data[key] != {}:
            if isinstance(data[key], list):  # If value is a list
                dataframe = pd.DataFrame(columns=flatten_json_to_row_column(flatten_json(data[key][0])).columns)
                for i in range(len(data[key])):
                    flatten_dict = flatten_json(data[key][i])
                    new_dataframe = flatten_json_to_row_column(flatten_dict)
                    dataframe = pd.concat([dataframe, new_dataframe], ignore_index=True)

                    # Deleting field translation columns
                    dataframe = dataframe.loc[:, ~dataframe.columns.str.contains("fieldtranslations|translation", case=False, regex=True)]

                if dataframe.shape[1] >= 10:  # If columns are 10 or more
                    all_dataframe[key] = dataframe
                else:
                    parent_json[key] = data[key]

            elif isinstance(data[key], dict):  # If value is a dictionary
                flatten_dict = flatten_json(data[key])
                dataframe = flatten_json_to_row_column(flatten_dict)

                # Deleting field translation columns
                dataframe = dataframe.loc[:, ~dataframe.columns.str.contains("fieldtranslations|translation", case=False, regex=True)]

                if dataframe.shape[1] >= 5:  # If columns are 5 or more
                    all_dataframe[key] = dataframe
                else:
                    parent_json[key] = data[key]
            else:
                parent_json[key] = data[key]

    # Convert parent_json into a DataFrame and store it in all_dataframe
    all_dataframe["match_stats"] = flatten_json_to_row_column(flatten_json(parent_json))

    return all_dataframe


# ************************************************************************************************
# API connection setup  
conn = http.client.HTTPSConnection("allsportsapi2.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "693b14936cmshb4229d5afd9d447p186b67jsn499928bbef0a",
    'x-rapidapi-host': "allsportsapi2.p.rapidapi.com"
}



# for each match id collecting data
for match_id in match_ids:
    
    ## ************************Data collection for match_details***************************
    try:
        # Make API request for match_details
        conn.request("GET", f"/api/cricket/match/{match_id}", headers=headers)
        res = conn.getresponse()
        data = res.read()
            
        # Convert response to JSON
        data_json = json.loads(data.decode("utf-8"))
        # converting data_json into row and columns dataframe
        match_details_data = process_json(data_json)['event']
        match_details = pd.concat([match_details, match_details_data], ignore_index=True)

    except Exception as e:
        print(f"Error fetching match_details for Match ID {match_id}: {e}")

    time.sleep(1)

    ## **************************Data collection for match_incidents****************************
    try:
        # Make API request for match_incidents
        conn.request("GET", f"/api/cricket/match/{match_id}/incidents", headers=headers)
        res = conn.getresponse()
        data = res.read()
            
        # Convert response to JSON
        data_json = json.loads(data.decode("utf-8"))
        # converting data_json into row and columns dataframe
        match_incidents_data = process_json(data_json)['incidents']
        # create a new column by match_id so that it can be unique identifier
        match_incidents_data['match_id'] = match_id
        match_incidents = pd.concat([match_incidents, match_incidents_data], ignore_index=True)

    except Exception as e:
        print(f"Error fetching match_incidents for Match ID {match_id}: {e}")

    time.sleep(1)

    ## ****************************Data collection for match_graph***************************
    try:
        # Make API request for match_graph
        conn.request("GET", f"/api/cricket/match/{match_id}/graph", headers=headers)
        res = conn.getresponse()
        data = res.read()
            
        # Convert response to JSON
        data_json = json.loads(data.decode("utf-8"))
        # converting data_json into row and columns dataframe
        homeinnings = flatten_json_to_row_column(data_json['graphPoints']['homeInnings'][0]['runs'])
        homeinnings['inning'] = 'home'
        homeinnings['match_id'] = match_id
        
        # awayinnings data_json extractiong
        awayinnings = flatten_json_to_row_column(data_json['graphPoints']['awayInnings'][0]['runs'])
        awayinnings['inning'] = 'away'
        awayinnings['match_id'] = match_id 

        # concatenating home and away inings data
        match_graph_data = pd.concat([homeinnings, awayinnings],ignore_index=True)
        # storing in match_graph dataframe
        match_graph = pd.concat([match_graph, match_graph_data], ignore_index=False)

    except Exception as e:
        print(f"Error fetching match_graph for Match ID {match_id}: {e}")

    time.sleep(1)


    ## ****************************Data collection for match_innings**************************
    try:
        # Make API request for match_innings
        conn.request("GET", f"/api/cricket/match/{match_id}/innings", headers=headers)
        res = conn.getresponse()
        data = res.read()
            
        # Convert response to JSON
        data_json = json.loads(data.decode("utf-8"))
        # converting data_json into row and columns dataframe

        # extracting innings1 and innings2 data separately from parent data
        innings1_dataframes = process_json_for_innings(data_json['innings'][0])
        innings2_dataframes = process_json_for_innings(data_json['innings'][1])

        # storing data properly
        # ************* For innings 1 *******
        # battingTeam
        innings1_battingTeam = innings1_dataframes['battingTeam']
        innings1_battingTeam['match_id'] = match_id
        innings1_battingTeam['innings'] = 1
        battingTeam = pd.concat([battingTeam, innings1_battingTeam], ignore_index=True)

        # bowlingTeam
        innings1_bowlingTeam = innings1_dataframes['bowlingTeam']
        innings1_bowlingTeam['match_id'] = match_id
        innings1_bowlingTeam['innings'] = 1
        bowlingTeam = pd.concat([bowlingTeam, innings1_bowlingTeam], ignore_index=True)

        # battingLine
        innings1_battingLine = innings1_dataframes['battingLine']
        innings1_battingLine['match_id'] = match_id
        innings1_battingLine['innings'] = 1
        battingLine = pd.concat([battingLine, innings1_battingLine], ignore_index=True)

        # bowlingLine
        innings1_bowlingLine = innings1_dataframes['bowlingLine']
        innings1_bowlingLine['match_id'] = match_id
        innings1_bowlingLine['innings'] = 1
        bowlingLine = pd.concat([bowlingLine, innings1_bowlingLine], ignore_index=True)

        # partnerships
        innings1_partnerships = innings1_dataframes['partnerships']
        innings1_partnerships['match_id'] = match_id
        innings1_partnerships['innings'] = 1
        partnerships = pd.concat([partnerships, innings1_partnerships], ignore_index=True)

        # match_stats
        innings1_match_stats = innings1_dataframes['match_stats']
        innings1_match_stats['match_id'] = match_id
        innings1_match_stats['innings'] = 1
        match_stats = pd.concat([match_stats, innings1_match_stats], ignore_index=True)


        # *********For innings 2 ******

        # battingTeam
        innings2_battingTeam = innings2_dataframes['battingTeam']
        innings2_battingTeam['match_id'] = match_id
        innings2_battingTeam['innings'] = 2
        battingTeam = pd.concat([battingTeam, innings2_battingTeam], ignore_index=True)

        # bowlingTeam
        innings2_bowlingTeam = innings2_dataframes['bowlingTeam']
        innings2_bowlingTeam['match_id'] = match_id
        innings2_bowlingTeam['innings'] = 2
        bowlingTeam = pd.concat([bowlingTeam, innings2_bowlingTeam], ignore_index=True)

        # battingLine
        innings2_battingLine = innings2_dataframes['battingLine']
        innings2_battingLine['match_id'] = match_id
        innings2_battingLine['innings'] = 2
        battingLine = pd.concat([battingLine, innings2_battingLine], ignore_index=True)


        # bowlingLine
        innings2_bowlingLine = innings2_dataframes['bowlingLine']
        innings2_bowlingLine['match_id'] = match_id
        innings2_bowlingLine['innings'] = 2
        bowlingLine = pd.concat([bowlingLine, innings2_bowlingLine], ignore_index=True)

        # partnerships
        innings2_partnerships = innings2_dataframes['partnerships']
        innings2_partnerships['match_id'] = match_id
        innings2_partnerships['innings'] = 2
        partnerships = pd.concat([partnerships, innings2_partnerships], ignore_index=True)

        # match_stats
        innings2_match_stats = innings2_dataframes['match_stats']
        innings2_match_stats['match_id'] = match_id
        innings2_match_stats['innings'] = 2
        match_stats = pd.concat([match_stats, innings2_match_stats], ignore_index=True)

    except Exception as e:
        print(f"Error fetching match_innings for Match ID {match_id}: {e}")

    time.sleep(1)
    
    # ************************* Data collection for Match Lineups **********************************
    try:
        # Make API request for match_lineups
        conn.request("GET", f"/api/cricket/match/{match_id}/lineups", headers=headers)
        res = conn.getresponse()
        data = res.read()
            
        # Convert response to JSON
        data_json = json.loads(data.decode("utf-8"))

        # Home lineups details
        home_lineups = process_json(data_json['home'])
        home_lineups['players']['match_id'] = match_id
        home_lineups['players']['innings'] = 'home'
        home_lineups['parent_df'] = home_lineups['parent_df'].loc[home_lineups['parent_df'].index.repeat(len(home_lineups['players']))].reset_index(drop=True) 
        home_lineups_details = pd.concat([home_lineups['players'], home_lineups['parent_df']], axis=1)

        # Away lineups details
        away_lineups = process_json(data_json['away'])
        away_lineups['players']['match_id'] = match_id
        away_lineups['players']['innings'] = 'away'
        away_lineups['parent_df'] = away_lineups['parent_df'].loc[away_lineups['parent_df'].index.repeat(len(away_lineups['players']))].reset_index(drop=True) 
        away_lineups_details = pd.concat([away_lineups['players'], away_lineups['parent_df']], axis=1)

        # Home and Away broth lineups details concatenating
        lineups_details = pd.concat([home_lineups_details, away_lineups_details], ignore_index=True)
        # inserting into match_lineups
        match_lineups = pd.concat([match_lineups, lineups_details], ignore_index=True)
    except Exception as e:
        print(f"Error fetching match_lineups for Match ID {match_id}: {e}")

    time.sleep(1)

    ## ***********************Data collection for match_odds**********************
    try:
        # Make API request for match_odds
        conn.request("GET", f"/api/cricket/match/{match_id}/odds", headers=headers)
        res = conn.getresponse()
        data = res.read()
            
        # Convert response to JSON
        data_json = json.loads(data.decode("utf-8"))
        # converting data_json into row and columns dataframe
        match_odds_details = process_json(data_json)['markets']
        match_odds_details['match_id'] = match_id
        match_odds = pd.concat([match_odds_details, match_odds], ignore_index=True)


    except Exception as e:
        print(f"Error fetching match_odds for Match ID {match_id}: {e}")

    time.sleep(1)

    ## ************************Data collection for match_duel***************************
    try:
        # Make API request for match_duel
        conn.request("GET", f"/api/cricket/match/{match_id}/duel", headers=headers)
        res = conn.getresponse()
        data = res.read()
            
        # Convert response to JSON
        data_json = json.loads(data.decode("utf-8"))
        # converting data_json into row and columns dataframe
        match_duel = flatten_json_to_row_column(flatten_json(data_json))
        match_duel['match_id'] = match_id
        match_h2h_duel = pd.concat([match_h2h_duel, match_duel], ignore_index=True)

    except Exception as e:
        print(f"Error fetching match_duel for Match ID {match_id}: {e}")

    time.sleep(1)



# Save all collected data  to an Excel file
file_path = "ICC championship Trophy 2025.xlsx"
with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
        match_details.to_excel(writer, sheet_name='match_details', index=False)
        match_incidents.to_excel(writer,sheet_name='match_incidents', index=False)
        match_graph.to_excel(writer,sheet_name='match_graph', index=False)
        match_stats.to_excel(writer,sheet_name='match_stats', index=False)
        battingTeam.to_excel(writer,sheet_name='battingTeam', index=False)
        bowlingTeam.to_excel(writer,sheet_name='bowlingTeam', index=False)
        battingLine.to_excel(writer,sheet_name='battingLine', index=False)
        bowlingLine.to_excel(writer,sheet_name='bowlingLine', index=False)
        partnerships.to_excel(writer,sheet_name='partnerships', index=False)
        match_lineups.to_excel(writer,sheet_name='match_lineups', index=False)
        match_h2h_duel.to_excel(writer,sheet_name='match_h2h_duel', index=False)
        match_odds.to_excel(writer,sheet_name='match_odds', index=False)

print(f"Excel file '{file_path}' saved successfully!")

C:\Program Files\KMSpico\temp\ipykernel_15912\1630972387.py:134: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, new_dataframe], ignore_index=True)
C:\Program Files\KMSpico\temp\ipykernel_15912\1630972387.py:134: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, new_dataframe], ignore_index=True)
C:\Program Files\KMSpico\temp\ipykernel_15912\1630972387.py:134: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA

In [51]:
match_details

,tournament_name,tournament_slug,tournament_category_id,tournament_category_name,tournament_category_slug,tournament_category_sport_name,tournament_category_sport_slug,tournament_category_sport_id,tournament_category_flag,tournament_uniqueTournament_name,...,cricketBallProperties_13,cricketBallProperties_14,cricketBallProperties_15,cricketBallProperties_16,cricketBallProperties_17,cricketBallProperties_18,cricketBallProperties_19,fanRatingEvent,showTotoPromo,isEditor
0,"ICC Champions Trophy, Group A",icc-champions-trophy-group-a,1343,World,world,Cricket,cricket,62,international,ICC Champions Trophy,...,commentary,id,ballDetails,zone,angle,bowlDetail,fielder,False,True,False


In [54]:
# Save to an Excel file
file_path = "ICC championship Trophy 2025.xlsx"
with pd.ExcelWriter(file_path, engine='xlsxwriter') as writer:
        match_details.to_excel(writer, sheet_name='match_details', index=False)
        match_incidents.to_excel(writer,sheet_name='match_incidents', index=False)
        match_graph.to_excel(writer,sheet_name='match_graph', index=False)
        match_stats.to_excel(writer,sheet_name='match_stats', index=False)
        battingTeam.to_excel(writer,sheet_name='battingTeam', index=False)
        bowlingTeam.to_excel(writer,sheet_name='bowlingTeam', index=False)
        battingLine.to_excel(writer,sheet_name='battingLine', index=False)
        bowlingLine.to_excel(writer,sheet_name='bowlingLine', index=False)
        partnerships.to_excel(writer,sheet_name='partnerships', index=False)
        match_lineups.to_excel(writer,sheet_name='match_lineups', index=False)
        match_h2h_duel.to_excel(writer,sheet_name='match_h2h_duel', index=False)
        match_odds.to_excel(writer,sheet_name='match_odds', index=False)

print(f"Excel file '{file_path}' saved successfully!")

Excel file 'ICC championship Trophy 2025.xlsx' saved successfully!


In [ ]:
match_lineups.to_excel(writer,sheet_name='match_lineups', index=False)

In [ ]:

# API connection setup
conn = http.client.HTTPSConnection("allsportsapi2.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "693b14936cmshb4229d5afd9d447p186b67jsn499928bbef0a",
    'x-rapidapi-host': "allsportsapi2.p.rapidapi.com"
}


## Data collection for match_details**********************************************
try:
    # Make API request for match_details
    conn.request("GET", f"/api/cricket/match/{match_id}", headers=headers)
    res = conn.getresponse()
    data = res.read()
        
    # Convert response to JSON
    data_json = json.loads(data.decode("utf-8"))
    # converting data_json into row and columns dataframe
    match_details_data = process_json(data_json)['event']
    match_details = pd.concat([match_details, match_details_data], ignore_index=True)

except Exception as e:
    print(f"Error fetching match_details for Match ID {match_id}: {e}")

time.sleep(1)


In [21]:
match_details

,tournament_name,tournament_slug,tournament_category_id,tournament_category_name,tournament_category_slug,tournament_category_sport_name,tournament_category_sport_slug,tournament_category_sport_id,tournament_category_flag,tournament_uniqueTournament_name,...,cricketBallProperties_13,cricketBallProperties_14,cricketBallProperties_15,cricketBallProperties_16,cricketBallProperties_17,cricketBallProperties_18,cricketBallProperties_19,fanRatingEvent,showTotoPromo,isEditor
0,"ICC Champions Trophy, Group A",icc-champions-trophy-group-a,1343,World,world,Cricket,cricket,62,international,ICC Champions Trophy,...,commentary,id,ballDetails,zone,angle,bowlDetail,fielder,False,True,False


In [22]:
match_id = 13265827
## Data collection for match_incidents**********************************************
try:
    # Make API request for match_incidents
    conn.request("GET", f"/api/cricket/match/{match_id}/incidents", headers=headers)
    res = conn.getresponse()
    data = res.read()
        
    # Convert response to JSON
    data_json = json.loads(data.decode("utf-8"))
    # converting data_json into row and columns dataframe
    match_incidents_data = process_json(data_json)['incidents']
    # create a new column by match_id so that it can be unique identifier
    match_incidents_data['match_id'] = match_id
    match_incidents = pd.concat([match_incidents, match_incidents_data], ignore_index=True)

except Exception as e:
    print(f"Error fetching match_incidents for Match ID {match_id}: {e}")

time.sleep(1)

In [23]:
match_incidents

,number,over,ball,runs,score,scored,wicket,missed,bowlDetail,batsman_name,...,fielder_jerseyNumber,fielder_height,fielder_userCount,fielder_id,fielder_marketValueCurrency,fielder_dateOfBirthTimestamp,fielder_cricketPlayerInfo_batting,fielder_cricketPlayerInfo_bowling,dismissedBatsman_height,match_id
0,302,48,2,0,260/10,False,True,False,Bowled,Naseem Shah,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13265827
1,301,48,1,0,260/9,False,False,False,NaN,Naseem Shah,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13265827
2,299,47,6,0,260/9,False,True,False,Caught,Haris Rauf,...,75,183.0,105.0,786833.0,EUR,674697600.0,Right,Right-arm medium,180.0,13265827
3,298,47,5,6,260/8,True,False,False,NaN,Haris Rauf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13265827
4,297,47,4,1,254/8,True,False,False,NaN,Naseem Shah,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13265827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,19,1,5,0,3/0,False,False,False,NaN,Devon Conway,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13265827
591,18,1,4,0,3/0,False,False,False,NaN,Devon Conway,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13265827
592,17,1,3,0,3/0,False,False,False,NaN,Devon Conway,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13265827
593,15,1,2,3,3/0,True,False,False,NaN,Will Young,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13265827


In [24]:
match_id = 13265827
## Data collection for match_graph**********************************************
try:
    # Make API request for match_graph
    conn.request("GET", f"/api/cricket/match/{match_id}/graph", headers=headers)
    res = conn.getresponse()
    data = res.read()
        
    # Convert response to JSON
    data_json = json.loads(data.decode("utf-8"))
    # converting data_json into row and columns dataframe
    homeinnings = flatten_json_to_row_column(data_json['graphPoints']['homeInnings'][0]['runs'])
    homeinnings['inning'] = 'home'
    homeinnings['match_id'] = match_id
    
    # awayinnings data_json extractiong
    awayinnings = flatten_json_to_row_column(data_json['graphPoints']['awayInnings'][0]['runs'])
    awayinnings['inning'] = 'away'
    awayinnings['match_id'] = match_id 

    # concatenating home and away inings data
    match_graph_data = pd.concat([homeinnings, awayinnings],ignore_index=True)
    # storing in match_graph dataframe
    match_graph = pd.concat([match_graph, match_graph_data], ignore_index=False)

except Exception as e:
    print(f"Error fetching match_graph for Match ID {match_id}: {e}")

time.sleep(1)

In [25]:
match_graph

,over,runs,wickets,inning,match_id
0,1,1,NaN,home,13265827
1,2,3,NaN,home,13265827
2,3,5,NaN,home,13265827
3,4,8,1.0,home,13265827
4,5,12,NaN,home,13265827
...,...,...,...,...,...
93,46,265,NaN,away,13265827
94,47,283,NaN,away,13265827
95,48,294,NaN,away,13265827
96,49,306,NaN,away,13265827


In [26]:
match_graph

,over,runs,wickets,inning,match_id
0,1,1,NaN,home,13265827
1,2,3,NaN,home,13265827
2,3,5,NaN,home,13265827
3,4,8,1.0,home,13265827
4,5,12,NaN,home,13265827
...,...,...,...,...,...
93,46,265,NaN,away,13265827
94,47,283,NaN,away,13265827
95,48,294,NaN,away,13265827
96,49,306,NaN,away,13265827


# Data collection for match_innings

In [27]:
def process_json_for_innings(data):
    all_dataframe = {}
    parent_json = {}

    for key in data:
        if data[key] is not None and data[key] != "" and data[key] != {}:
            if isinstance(data[key], list):  # If value is a list
                dataframe = pd.DataFrame(columns=flatten_json_to_row_column(flatten_json(data[key][0])).columns)
                for i in range(len(data[key])):
                    flatten_dict = flatten_json(data[key][i])
                    new_dataframe = flatten_json_to_row_column(flatten_dict)
                    dataframe = pd.concat([dataframe, new_dataframe], ignore_index=True)

                    # Deleting field translation columns
                    dataframe = dataframe.loc[:, ~dataframe.columns.str.contains("fieldtranslations|translation", case=False, regex=True)]

                if dataframe.shape[1] >= 10:  # If columns are 10 or more
                    all_dataframe[key] = dataframe
                else:
                    parent_json[key] = data[key]

            elif isinstance(data[key], dict):  # If value is a dictionary
                flatten_dict = flatten_json(data[key])
                dataframe = flatten_json_to_row_column(flatten_dict)

                # Deleting field translation columns
                dataframe = dataframe.loc[:, ~dataframe.columns.str.contains("fieldtranslations|translation", case=False, regex=True)]

                if dataframe.shape[1] >= 5:  # If columns are 5 or more
                    all_dataframe[key] = dataframe
                else:
                    parent_json[key] = data[key]
            else:
                parent_json[key] = data[key]

    # Convert parent_json into a DataFrame and store it in all_dataframe
    all_dataframe["match_stats"] = flatten_json_to_row_column(flatten_json(parent_json))

    return all_dataframe

In [ ]:

## Data collection for match_innings**********************************************
try:
    # Make API request for match_innings
    conn.request("GET", f"/api/cricket/match/{match_id}/innings", headers=headers)
    res = conn.getresponse()
    data = res.read()
        
    # Convert response to JSON
    data_json = json.loads(data.decode("utf-8"))
    # converting data_json into row and columns dataframe
    # extracting innings1 and innings2 data separately from parent data
    innings1_dataframes = process_json_for_innings(data_json['innings'][0])
    innings2_dataframes = process_json_for_innings(data_json['innings'][1])

    # storing data properly
    # ******************************** For innings 1 ************************************
    # battingTeam
    innings1_battingTeam = innings1_dataframes['battingTeam']
    innings1_battingTeam['match_id'] = match_id
    innings1_battingTeam['innings'] = 1
    battingTeam = pd.concat([battingTeam, innings1_battingTeam], ignore_index=True)

    # bowlingTeam
    innings1_bowlingTeam = innings1_dataframes['bowlingTeam']
    innings1_bowlingTeam['match_id'] = match_id
    innings1_bowlingTeam['innings'] = 1
    bowlingTeam = pd.concat([bowlingTeam, innings1_bowlingTeam], ignore_index=True)

    # battingLine
    innings1_battingLine = innings1_dataframes['battingLine']
    innings1_battingLine['match_id'] = match_id
    innings1_battingLine['innings'] = 1
    battingLine = pd.concat([battingLine, innings1_battingLine], ignore_index=True)

    # bowlingLine
    innings1_bowlingLine = innings1_dataframes['bowlingLine']
    innings1_bowlingLine['match_id'] = match_id
    innings1_bowlingLine['innings'] = 1
    bowlingLine = pd.concat([bowlingLine, innings1_bowlingLine], ignore_index=True)

    # partnerships
    innings1_partnerships = innings1_dataframes['partnerships']
    innings1_partnerships['match_id'] = match_id
    innings1_partnerships['innings'] = 1
    partnerships = pd.concat([partnerships, innings1_partnerships], ignore_index=True)

    # match_stats
    innings1_match_stats = innings1_dataframes['match_stats']
    innings1_match_stats['match_id'] = match_id
    innings1_match_stats['innings'] = 1
    match_stats = pd.concat([match_stats, innings1_match_stats], ignore_index=True)


    # ***************************** For innings 2 *********************************

    # battingTeam
    innings2_battingTeam = innings2_dataframes['battingTeam']
    innings2_battingTeam['match_id'] = match_id
    innings2_battingTeam['innings'] = 2
    battingTeam = pd.concat([battingTeam, innings2_battingTeam], ignore_index=True)

    # bowlingTeam
    innings2_bowlingTeam = innings2_dataframes['bowlingTeam']
    innings2_bowlingTeam['match_id'] = match_id
    innings2_bowlingTeam['innings'] = 2
    bowlingTeam = pd.concat([bowlingTeam, innings2_bowlingTeam], ignore_index=True)

    # battingLine
    innings2_battingLine = innings2_dataframes['battingLine']
    innings2_battingLine['match_id'] = match_id
    innings2_battingLine['innings'] = 2
    battingLine = pd.concat([battingLine, innings2_battingLine], ignore_index=True)


    # bowlingLine
    innings2_bowlingLine = innings2_dataframes['bowlingLine']
    innings2_bowlingLine['match_id'] = match_id
    innings2_bowlingLine['innings'] = 2
    bowlingLine = pd.concat([bowlingLine, innings2_bowlingLine], ignore_index=True)

    # partnerships
    innings2_partnerships = innings2_dataframes['partnerships']
    innings2_partnerships['match_id'] = match_id
    innings2_partnerships['innings'] = 2
    partnerships = pd.concat([partnerships, innings2_partnerships], ignore_index=True)

    # match_stats
    innings2_match_stats = innings2_dataframes['match_stats']
    innings2_match_stats['match_id'] = match_id
    innings2_match_stats['innings'] = 2
    match_stats = pd.concat([match_stats, innings2_match_stats], ignore_index=True)

except Exception as e:
    print(f"Error fetching match_innings for Match ID {match_id}: {e}")

time.sleep(1)

C:\Program Files\KMSpico\temp\ipykernel_15912\252130191.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, new_dataframe], ignore_index=True)
C:\Program Files\KMSpico\temp\ipykernel_15912\252130191.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, new_dataframe], ignore_index=True)
C:\Program Files\KMSpico\temp\ipykernel_15912\252130191.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entri

In [29]:
data_json

{'innings': [{'number': 1,
   'battingTeam': {'name': 'New Zealand',
    'slug': 'new-zealand',
    'shortName': 'NZ',
    'gender': 'M',
    'sport': {'name': 'Cricket', 'slug': 'cricket', 'id': 62},
    'userCount': 14663,
    'nameCode': 'NZE',
    'disabled': False,
    'national': True,
    'type': 0,
    'id': 187755,
    'entityType': 'team',
    'teamColors': {'primary': '#ffffff',
     'secondary': '#000000',
     'text': '#000000'},
    'fieldTranslations': {'nameTranslation': {'ar': 'نيوزيلندا',
      'hi': 'न्यूज़ीलैंड',
      'bn': 'নিউজিল্যান্ড'},
     'shortNameTranslation': {'ar': 'نيوزيلندا',
      'hi': 'एनज़ेड',
      'bn': 'এনজেড'}}},
   'bowlingTeam': {'name': 'Pakistan',
    'slug': 'pakistan',
    'shortName': 'PAK',
    'gender': 'M',
    'sport': {'name': 'Cricket', 'slug': 'cricket', 'id': 62},
    'userCount': 20392,
    'nameCode': 'PAK',
    'disabled': False,
    'national': True,
    'type': 0,
    'id': 187754,
    'entityType': 'team',
    'teamColors':

In [ ]:
# Data Collection for Match Lineups

conn = http.client.HTTPSConnection("allsportsapi2.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "693b14936cmshb4229d5afd9d447p186b67jsn499928bbef0a",
    'x-rapidapi-host': "allsportsapi2.p.rapidapi.com"
}
try:
    # Make API request for match_lineups
    conn.request("GET", f"/api/cricket/match/{match_id}/lineups", headers=headers)
    res = conn.getresponse()
    data = res.read()
        
    # Convert response to JSON
    data_json = json.loads(data.decode("utf-8"))

    # Home lineups details
    home_lineups = process_json(data_json['home'])
    home_lineups['players']['match_id'] = match_id
    home_lineups['players']['innings'] = 'home'
    home_lineups['parent_df'] = home_lineups['parent_df'].loc[home_lineups['parent_df'].index.repeat(len(home_lineups['players']))].reset_index(drop=True) 
    home_lineups_details = pd.concat([home_lineups['players'], home_lineups['parent_df']], axis=1)

    # Away lineups details
    away_lineups = process_json(data_json['away'])
    away_lineups['players']['match_id'] = match_id
    away_lineups['players']['innings'] = 'away'
    away_lineups['parent_df'] = away_lineups['parent_df'].loc[away_lineups['parent_df'].index.repeat(len(away_lineups['players']))].reset_index(drop=True) 
    away_lineups_details = pd.concat([away_lineups['players'], away_lineups['parent_df']], axis=1)

    # Home and Away broth lineups details concatenating
    lineups_details = pd.concat([home_lineups_details, away_lineups_details], ignore_index=True)
    # inserting into match_lineups
    match_lineups = pd.concat([match_lineups, lineups_details], ignore_index=True)
except Exception as e:
    print(f"Error fetching match_lineups for Match ID {match_id}: {e}")

time.sleep(1)


In [31]:
match_lineups

,player_name,player_slug,player_shortName,player_position,player_userCount,player_id,player_country_alpha2,player_country_alpha3,player_country_name,player_country_slug,...,supportStaff_1_country_alpha3,supportStaff_1_country_name,supportStaff_1_country_slug,supportStaff_2_name,supportStaff_2_role,supportStaff_2_id,supportStaff_2_country_alpha2,supportStaff_2_country_alpha3,supportStaff_2_country_name,supportStaff_2_country_slug
0,Abrar Ahmed,abrar-ahmed,A Ahmed,B,33,871548,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Babar Azam,babar-azam,B Azam,BM,1772,794647,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fakhar Zaman,fakhar-zaman,Fakhar Zaman,BM,206,795320,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Haris Rauf,haris-rauf,H Rauf,B,104,975942,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Khushdil Shah,khushdil-shah,K Shah,AR,26,874504,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Naseem Shah,naseem-shah,N Shah,B,229,956739,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Salman Agha,agha-salman,S Agha,AR,68,786840,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Saud Shakeel,saud-shakeel,S Shakeel,BM,28,975996,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Shaheen Afridi,shaheen-afridi,S Afridi,B,390,911674,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Tayyab Tahir,tayyab-tahir,T Tahir,BM,12,1212744,PK,PAK,Pakistan,pakistan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Data Collection for match_odds
# API connection setup
conn = http.client.HTTPSConnection("allsportsapi2.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "693b14936cmshb4229d5afd9d447p186b67jsn499928bbef0a",
    'x-rapidapi-host': "allsportsapi2.p.rapidapi.com"
}


## Data collection for match_odds**********************************************
try:
    # Make API request for match_odds
    conn.request("GET", f"/api/cricket/match/{match_id}/odds", headers=headers)
    res = conn.getresponse()
    data = res.read()
        
    # Convert response to JSON
    data_json = json.loads(data.decode("utf-8"))
    # converting data_json into row and columns dataframe
    match_odds_details = process_json(data_json)['markets']
    match_odds_details['match_id'] = match_id
    match_odds = pd.concat([match_odds_details, match_odds], ignore_index=True)


except Exception as e:
    print(f"Error fetching match_odds for Match ID {match_id}: {e}")

time.sleep(1)

In [33]:
match_odds

,sourceId,structureType,marketId,marketName,isLive,fid,suspended,id,marketGroup,marketPeriod,...,choices_1_sourceId,choices_1_name,choices_1_winning,choices_1_change,choices_2_initialFractionalValue,choices_2_fractionalValue,choices_2_sourceId,choices_2_name,choices_2_change,match_id
0,169647244,1,1,Full time,False,169647244,False,1984214393,1X2,Full time,...,96527028,2,True,1,NaN,NaN,NaN,NaN,NaN,13265827
1,169647244,1,26,To win the toss,False,169647244,False,1984252581,To win the toss,Full time,...,96557233,2,NaN,0,NaN,NaN,NaN,NaN,NaN,13265827
2,169647244,1,30,A hundred to be scored in match,False,169647244,False,1984252583,A hundred to be scored in match,Full time,...,96557548,No,NaN,-1,NaN,NaN,NaN,NaN,NaN,13265827
3,169647244,1,31,Most match sixes,False,169647244,False,1984252582,Most match sixes,Full time,...,96557186,X,NaN,0,6/5,10/11,96557189.0,2,-1.0,13265827
4,169647244,1,32,A fifty to be scored in the match,False,169647244,False,1984252584,A fifty to be scored in the match,Full time,...,96557475,No,NaN,0,NaN,NaN,NaN,NaN,NaN,13265827


In [34]:
# Match H2H duel Data collection
# Data Collection for match_duel
match_id = 13265827
# API connection setup
conn = http.client.HTTPSConnection("allsportsapi2.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "693b14936cmshb4229d5afd9d447p186b67jsn499928bbef0a",
    'x-rapidapi-host': "allsportsapi2.p.rapidapi.com"
}


## Data collection for match_duel**********************************************
try:
    # Make API request for match_duel
    conn.request("GET", f"/api/cricket/match/{match_id}/duel", headers=headers)
    res = conn.getresponse()
    data = res.read()
        
    # Convert response to JSON
    data_json = json.loads(data.decode("utf-8"))
    # converting data_json into row and columns dataframe
    match_duel = flatten_json_to_row_column(flatten_json(data_json))
    match_duel['match_id'] = match_id
    match_h2h_duel = pd.concat([match_h2h_duel, match_duel], ignore_index=True)

except Exception as e:
    print(f"Error fetching match_duel for Match ID {match_id}: {e}")

time.sleep(1)


In [45]:
match_odds

,sourceId,structureType,marketId,marketName,isLive,fid,suspended,id,marketGroup,marketPeriod,...,choices_1_sourceId,choices_1_name,choices_1_winning,choices_1_change,choices_2_initialFractionalValue,choices_2_fractionalValue,choices_2_sourceId,choices_2_name,choices_2_change,match_id
0,169647244,1,1,Full time,False,169647244,False,1984214393,1X2,Full time,...,96527028,2,True,1,NaN,NaN,NaN,NaN,NaN,13265827
1,169647244,1,26,To win the toss,False,169647244,False,1984252581,To win the toss,Full time,...,96557233,2,NaN,0,NaN,NaN,NaN,NaN,NaN,13265827
2,169647244,1,30,A hundred to be scored in match,False,169647244,False,1984252583,A hundred to be scored in match,Full time,...,96557548,No,NaN,-1,NaN,NaN,NaN,NaN,NaN,13265827
3,169647244,1,31,Most match sixes,False,169647244,False,1984252582,Most match sixes,Full time,...,96557186,X,NaN,0,6/5,10/11,96557189.0,2,-1.0,13265827
4,169647244,1,32,A fifty to be scored in the match,False,169647244,False,1984252584,A fifty to be scored in the match,Full time,...,96557475,No,NaN,0,NaN,NaN,NaN,NaN,NaN,13265827
